In [2]:
from timeit import default_timer as timer
import numpy as np
import pandas as pd

dataset_path = '/kaggle/input/intern-data/intern_task.csv'

In [6]:
main_data = pd.read_csv(dataset_path)
main_data

,rank,query_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_134,feature_135,feature_136,feature_137,feature_138,feature_139,feature_140,feature_141,feature_142,feature_143
0,0,10,1.0,0.0,1.0,3.0,3.0,0.333333,0.0,0.333333,...,0.0,0.000000,0.454545,0.890238,8.655534,1.000000,0.077778,0.002222,1.0,0.333333
1,1,10,3.0,0.0,3.0,0.0,3.0,1.000000,0.0,1.000000,...,0.0,0.000000,0.000000,0.773976,23.130514,0.000000,0.027826,0.000430,44.0,14.666667
2,0,10,3.0,0.0,2.0,0.0,3.0,1.000000,0.0,0.666667,...,0.0,0.000000,0.000000,0.918308,13.351339,0.000000,0.014925,0.000104,22.0,7.333333
3,1,10,3.0,0.0,3.0,0.0,3.0,1.000000,0.0,1.000000,...,0.0,0.000000,0.000000,0.975355,18.240926,0.000000,0.053140,0.000255,8.0,2.666667
4,2,10,3.0,0.0,3.0,1.0,3.0,1.000000,0.0,1.000000,...,273.0,79.670665,0.200000,0.990119,31.786048,0.333333,0.046512,0.000307,24.0,8.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235253,2,29995,1.0,0.0,0.0,0.0,1.0,0.500000,0.0,0.000000,...,0.0,0.000000,0.000000,0.471409,0.000000,0.000000,0.001350,0.000002,3.0,1.500000
235254,2,29995,1.0,0.0,1.0,0.0,1.0,0.500000,0.0,0.500000,...,0.0,0.000000,0.000000,0.471409,39.908056,0.000000,0.004850,0.000014,9.0,4.500000
235255,1,29995,1.0,0.0,0.0,0.0,1.0,0.500000,0.0,0.000000,...,0.0,0.000000,0.000000,0.471409,0.000000,0.000000,0.001064,0.000001,1.0,0.500000
235256,2,29995,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000


Разделим датасет на 3 выборки. На валидацию и тест отнесем примерно по 20 процентов запросов.

In [7]:
quids = main_data['query_id'].unique()

In [8]:
len(quids)

2000

In [9]:
first_sep = quids[round(len(quids)*0.6)]
first_sep

18010

In [10]:
second_sep = quids[round(len(quids)*0.8)]
second_sep

24010

In [11]:
train = main_data[main_data['query_id'] < first_sep]
val = main_data[(first_sep <= main_data['query_id']) & (main_data['query_id'] < second_sep)]
test = main_data[second_sep <= main_data['query_id']]

In [12]:
train

,rank,query_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_134,feature_135,feature_136,feature_137,feature_138,feature_139,feature_140,feature_141,feature_142,feature_143
0,0,10,1.0,0.0,1.0,3.0,3.0,0.333333,0.0,0.333333,...,0.0,0.000000,0.454545,0.890238,8.655534,1.000000,0.077778,0.002222,1.0,0.333333
1,1,10,3.0,0.0,3.0,0.0,3.0,1.000000,0.0,1.000000,...,0.0,0.000000,0.000000,0.773976,23.130514,0.000000,0.027826,0.000430,44.0,14.666667
2,0,10,3.0,0.0,2.0,0.0,3.0,1.000000,0.0,0.666667,...,0.0,0.000000,0.000000,0.918308,13.351339,0.000000,0.014925,0.000104,22.0,7.333333
3,1,10,3.0,0.0,3.0,0.0,3.0,1.000000,0.0,1.000000,...,0.0,0.000000,0.000000,0.975355,18.240926,0.000000,0.053140,0.000255,8.0,2.666667
4,2,10,3.0,0.0,3.0,1.0,3.0,1.000000,0.0,1.000000,...,273.0,79.670665,0.200000,0.990119,31.786048,0.333333,0.046512,0.000307,24.0,8.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143936,0,17995,0.0,0.0,0.0,2.0,2.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.153846,0.000000,0.000000,1.000000,0.002786,0.000000,0.0,0.000000
143937,0,17995,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
143938,1,17995,2.0,0.0,2.0,1.0,2.0,1.000000,0.0,1.000000,...,0.0,0.000000,0.111111,1.000000,27.835708,0.500000,0.027094,0.000000,30.0,15.000000
143939,0,17995,2.0,0.0,2.0,2.0,2.0,1.000000,0.0,1.000000,...,0.0,0.000000,0.200000,0.991224,27.835708,1.000000,0.022333,0.000007,14.0,7.000000


In [13]:
val

,rank,query_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_134,feature_135,feature_136,feature_137,feature_138,feature_139,feature_140,feature_141,feature_142,feature_143
143941,1,18010,2.0,0.0,2.0,1.0,2.0,1.0,0.000000,1.000000,...,0.0,0.0,0.111111,0.996212,31.770786,0.500000,0.058036,0.000027,9.0,4.500000
143942,1,18010,2.0,0.0,2.0,0.0,2.0,1.0,0.000000,1.000000,...,0.0,0.0,0.000000,0.997534,20.005554,0.000000,0.014797,0.000002,22.0,11.000000
143943,0,18010,2.0,0.0,2.0,1.0,2.0,1.0,0.000000,1.000000,...,0.0,0.0,0.181818,0.875081,40.011107,0.500000,0.073171,0.000816,12.0,6.000000
143944,0,18010,2.0,0.0,2.0,1.0,2.0,1.0,0.000000,1.000000,...,0.0,0.0,0.166667,0.954119,20.005554,0.500000,0.059091,0.000255,9.0,4.500000
143945,2,18010,2.0,0.0,2.0,1.0,2.0,1.0,0.000000,1.000000,...,0.0,0.0,0.181818,0.971592,40.011107,0.500000,0.072650,0.000234,11.0,5.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198155,2,23995,3.0,3.0,3.0,0.0,3.0,1.0,1.000000,1.000000,...,0.0,0.0,0.000000,1.000000,17.985703,0.000000,0.008787,0.000000,9.0,3.000000
198156,1,23995,3.0,0.0,2.0,0.0,3.0,1.0,0.000000,0.666667,...,0.0,0.0,0.000000,0.675582,10.379662,0.000000,0.023293,0.000225,143.0,47.666667
198157,0,23995,3.0,1.0,2.0,1.0,3.0,1.0,0.333333,0.666667,...,0.0,0.0,0.100000,0.891748,10.379662,0.333333,0.009378,0.000024,18.0,6.000000
198158,0,23995,3.0,3.0,3.0,3.0,3.0,1.0,1.000000,1.000000,...,0.0,0.0,0.230769,0.986125,17.985703,1.000000,0.026897,0.000009,19.0,6.333333


In [14]:
test

,rank,query_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_134,feature_135,feature_136,feature_137,feature_138,feature_139,feature_140,feature_141,feature_142,feature_143
198160,0,24010,3.0,2.0,2.0,0.0,3.0,1.000000,0.666667,0.666667,...,0.0,0.0,0.000000,0.805907,16.473262,0.000000,0.013168,0.000073,31.0,10.333333
198161,1,24010,2.0,0.0,2.0,2.0,2.0,0.666667,0.000000,0.666667,...,0.0,0.0,0.222222,0.776164,39.768988,0.666667,0.027168,0.000278,19.0,6.333333
198162,1,24010,2.0,0.0,1.0,1.0,2.0,0.666667,0.000000,0.333333,...,0.0,0.0,0.100000,0.779020,6.822464,0.333333,0.009606,0.000041,8.0,2.666667
198163,0,24010,3.0,2.0,2.0,0.0,3.0,1.000000,0.666667,0.666667,...,0.0,0.0,0.000000,0.824919,26.124061,0.000000,0.015705,0.000088,35.0,11.666667
198164,0,24010,3.0,1.0,0.0,0.0,3.0,1.000000,0.333333,0.000000,...,3.0,29.9,0.000000,0.740175,0.000000,0.000000,0.004230,0.000019,56.0,18.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235253,2,29995,1.0,0.0,0.0,0.0,1.0,0.500000,0.000000,0.000000,...,0.0,0.0,0.000000,0.471409,0.000000,0.000000,0.001350,0.000002,3.0,1.500000
235254,2,29995,1.0,0.0,1.0,0.0,1.0,0.500000,0.000000,0.500000,...,0.0,0.0,0.000000,0.471409,39.908056,0.000000,0.004850,0.000014,9.0,4.500000
235255,1,29995,1.0,0.0,0.0,0.0,1.0,0.500000,0.000000,0.000000,...,0.0,0.0,0.000000,0.471409,0.000000,0.000000,0.001064,0.000001,1.0,0.500000
235256,2,29995,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000


Будем считать NDCG, поэтому переведем разметку в интервал [0;1]:

In [15]:
def normalize_(vector: pd.Series) -> np.ndarray:
    max_relevance = np.max(vector)
    return (vector / max_relevance).values

Удалим разметку из тестовой части:

In [16]:
y_true = test['rank']
test.drop(['rank'], axis=1, inplace=True)

/tmp/ipykernel_34/1129453403.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.drop(['rank'], axis=1, inplace=True)


Будем обучать Catboost, а для подбора параметров воспользуемся hyperopt:

In [17]:
!pip install hyperopt==0.2.5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 966.0/966.0 kB 11.4 MB/s eta 0:00:0000:010:01
  Attempting uninstall: hyperopt
    Found existing installation: hyperopt 0.2.7
    Uninstalling hyperopt-0.2.7:
      Successfully uninstalled hyperopt-0.2.7


Для удобства оформим всё в виде отдельного класса:

In [60]:
from catboost import CatBoostRanker, Pool, utils
import hyperopt
from numpy.random import RandomState

class Model:
    def __init__(self):
        self.best = None
    
    def hyperopt_objective(self, params):
        '''
        Функция, которую будет оптимизировать hyperopt
        '''
        print(params)
        model = CatBoostRanker(
            **params,
            random_seed=42,
            verbose=False,
            task_type='GPU',
            iterations=5000
        )
        model.fit(self.train, eval_set=self.valid)
        score = utils.eval_metric(self.valid.get_label(), model.predict(self.valid), 
                                  'NDCG:top=5;type=Exp', group_id=self.valid.get_group_id_hash())[0]
        return 1 - score

    @staticmethod
    def normalize_(vector: pd.Series) -> pd.Series:
        '''
        Будем считать NDCG, поэтому переведем разметку в интервал [0;1] с помощью этого метода
        '''
        max_relevance = np.max(vector)
        return vector / max_relevance

    def params_optimize(self, df_train, df_valid, method):
        '''
        Метод для подбора параметров
        '''
        y_train = Model.normalize_(df_train['rank'])
        y_valid = Model.normalize_(df_valid['rank'])
        df_train_proc = df_train.drop(['rank', 'query_id'], axis=1)
        df_valid_proc = df_valid.drop(['rank', 'query_id'], axis=1)

        queries_train = df_train['query_id'].astype('Int32')
        queries_test = df_valid['query_id'].astype('Int32')

        train = Pool(
            data=df_train_proc,
            label=y_train,
            group_id=queries_train
        )

        test = Pool(
            data=df_valid_proc,
            label=y_valid,
            group_id=queries_test
        )

        self.train = train
        self.valid = test
        #сетка выбрана на основе https://github.com/catboost/benchmarks/blob/master/ranking/Readme.md
        params_space = {
            'learning_rate': hyperopt.hp.uniform('learning_rate', 0.01, 0.3),
            'depth': hyperopt.hp.quniform('depth', 4, 12, 1),
            'l2_leaf_reg': hyperopt.hp.uniform('l2_leaf_reg', 1, 20),
            'bagging_temperature': hyperopt.hp.uniform('bagging_temperature', 0.8, 1.2),
            'eval_metric': 'NDCG:top=10;type=Exp',
            'loss_function': 'YetiRank',
            'grow_policy': hyperopt.hp.choice('grow_policy', ['Depthwise', 'SymmetricTree', 'Lossguide']),
            'min_data_in_leaf': hyperopt.hp.quniform('min_data_in_leaf', 50, 100, 1),
            'max_bin': hyperopt.hp.quniform('max_bin', 64, 254, 1)
        }

        trials = hyperopt.Trials()
        
        best = hyperopt.fmin(
            self.hyperopt_objective,
            space=params_space,
            algo=method,
            max_evals=60,
            trials=trials,
            rstate=RandomState(42)            
        )
        self.best = best
        self.best['grow_policy'] = ['Depthwise', 'SymmetricTree', 'Lossguide'][self.best['grow_policy']]
        return self.best

    def fit(self, df_train, df_valid, device='CPU'):
        '''
        Итоговое обучение модели. Параметры подхватываются из атрибута best, если заранее были подобраны.
        '''
        y_train = Model.normalize_(df_train['rank'])
        y_valid = Model.normalize_(df_valid['rank'])
        df_train_proc = df_train.drop(['rank', 'query_id'], axis=1)
        df_valid_proc = df_valid.drop(['rank', 'query_id'], axis=1)

        queries_train = df_train['query_id'].astype('Int32')
        queries_test = df_valid['query_id'].astype('Int32')

        train = Pool(
            data=df_train_proc,
            label=y_train,
            group_id=queries_train,
        )

        test = Pool(
            data=df_valid_proc,
            label=y_valid,
            group_id=queries_test
        )

        self.train = train
        self.valid = test

        if self.best:
            self.model = CatBoostRanker(
                **self.best,
                eval_metric='NDCG:top=5;type=Exp',
                random_seed=42,
                verbose=False,
                loss_function='YetiRank',
                iterations=15000, #берём с запасом, в конце возьмем модель с лучшей метрикой на валидации
                use_best_model=True,
                task_type=device
            )
        else:
            print('Используем параметры по умолчанию')
            self.model = CatBoostRanker(
                eval_metric='NDCG:top=5;type=Exp',
                random_seed=42,
                verbose=False,
                loss_function='YetiRank',
                iterations=15000, #берём с запасом, в конце возьмем модель с лучшей метрикой на валидации
                use_best_model=True
            )

        self.model.fit(train, eval_set=test, plot=True)


    def predict(self, df_test):
        return self.model.predict(df_test)

Подберем параметры. Делать это будем на GPU, потому что так сильно быстрее. Прогоним 60 итераций оптимизации по 5000 итераций бустинга в каждой. Будем использовать Байесовский метод оптимизации:

In [43]:
model = Model()

opt_params = model.params_optimize(train, val, hyperopt.tpe.suggest)
opt_params

{'bagging_temperature': 1.0045605546606369, 'depth': 10.0, 'eval_metric': 'NDCG:top=10;type=Exp', 'grow_policy': 'Depthwise', 'l2_leaf_reg': 7.069497769755579, 'learning_rate': 0.03002488885334522, 'loss_function': 'YetiRank', 'max_bin': 222.0, 'min_data_in_leaf': 73.0}
  0%|          | 0/60 [00:00<?, ?trial/s, best loss=?]

Default metric period is 5 because PFound
, NDCG
 is/are not implemented for GPU

Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time

Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time



{'bagging_temperature': 1.1898703853813748, 'depth': 4.0, 'eval_metric': 'NDCG:top=10;type=Exp', 'grow_policy': 'Lossguide', 'l2_leaf_reg': 7.000192276253925, 'learning_rate': 0.28053171966587176, 'loss_function': 'YetiRank', 'max_bin': 202.0, 'min_data_in_leaf': 71.0}
  2%|▏         | 1/60 [02:25<2:23:16, 145.70s/trial, best loss: 0.4472804172870696]

Default metric period is 5 because PFound
, NDCG
 is/are not implemented for GPU

Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time

Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time



{'bagging_temperature': 0.9380358700105802, 'depth': 8.0, 'eval_metric': 'NDCG:top=10;type=Exp', 'grow_policy': 'Depthwise', 'l2_leaf_reg': 19.073254225273256, 'learning_rate': 0.09798893186641072, 'loss_function': 'YetiRank', 'max_bin': 232.0, 'min_data_in_leaf': 92.0}
  3%|▎         | 2/60 [03:20<1:28:58, 92.04s/trial, best loss: 0.44070925622328316]

...

Default metric period is 5 because PFound
, NDCG
 is/are not implemented for GPU

Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time

Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time



{'bagging_temperature': 1.1973737548392895, 'depth': 4.0, 'eval_metric': 'NDCG:top=10;type=Exp', 'grow_policy': 'Depthwise', 'l2_leaf_reg': 2.4788550219485272, 'learning_rate': 0.12453649697061611, 'loss_function': 'YetiRank', 'max_bin': 81.0, 'min_data_in_leaf': 92.0}
 98%|█████████▊| 59/60 [1:15:19<01:08, 68.72s/trial, best loss: 0.4356587409209951]

Default metric period is 5 because PFound
, NDCG
 is/are not implemented for GPU

Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time

Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time



100%|██████████| 60/60 [1:16:07<00:00, 76.13s/trial, best loss: 0.4356587409209951]


{'bagging_temperature': 1.1147403275958099,
 'depth': 5.0,
 'grow_policy': 'Depthwise',
 'l2_leaf_reg': 4.139438168606409,
 'learning_rate': 0.2183714035697752,
 'max_bin': 75.0,
 'min_data_in_leaf': 53.0}

Обучим итоговую модель:

In [44]:

# Fit
start = timer()
model.fit(train, val, device='GPU')
elapsed = timer() - start
print(f"Model fit: elapsed = {elapsed:.3f}")


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Default metric period is 5 because PFound, NDCG is/are not implemented for GPU
Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=5;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


Model fit: elapsed = 157.799


In [45]:
# Predict
y_hat_test = model.predict(test)
print(f"Predicted: y_hat_test.shape = {y_hat_test.shape}")

Predicted: y_hat_test.shape = (37098,)


In [46]:
def compute_metrics(y_true, y_hat, q):
    # List of metrics to evaluate
    eval_metrics = ['NDCG:top=5;type=Exp', 'NDCG:top=10;type=Exp']
    
    for eval_metric in eval_metrics:
        scores = utils.eval_metric(y_true, y_hat, eval_metric, group_id=q)
    
        # Print scores
        print(f"metric = {eval_metric} score = {scores[0]:.3f}")

# Get test targets and groups
y_test = y_true.to_numpy()
q_test = test['query_id'].to_numpy().astype('uint32')
    
# Compute metrics on test
compute_metrics(y_test, y_hat_test, q_test)

metric = NDCG:top=5;type=Exp score = 0.461
metric = NDCG:top=10;type=Exp score = 0.494


В качестве итогового набора оставим следующий:
```python
{'bagging_temperature': 1.1147403275958099,
 'depth': 5.0,
 'grow_policy': 'Depthwise',
 'l2_leaf_reg': 4.139438168606409,
 'learning_rate': 0.2183714035697752,
 'max_bin': 75.0,
 'min_data_in_leaf': 53.0}
```

Обучим финальную модель с этим набором и сохраним в файл:

In [63]:
params = {'bagging_temperature': 1.1147403275958099,
 'depth': 5.0,
 'grow_policy': 'Depthwise',
 'l2_leaf_reg': 4.139438168606409,
 'learning_rate': 0.2183714035697752,
 'max_bin': 75.0,
 'min_data_in_leaf': 53.0}

model = Model()
model.best = params

start = timer()
model.fit(train, val, device='GPU')
elapsed = timer() - start
print(f"Model fit: elapsed = {elapsed:.3f}")
y_hat_test = model.predict(test)
print(f"Predicted: y_hat_test.shape = {y_hat_test.shape}")

    
# Compute metrics on test
compute_metrics(y_test, y_hat_test, q_test)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Default metric period is 5 because PFound, NDCG is/are not implemented for GPU
Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=5;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


Model fit: elapsed = 158.866
Predicted: y_hat_test.shape = (37098,)
metric = NDCG:top=5;type=Exp score = 0.465
metric = NDCG:top=10;type=Exp score = 0.494


Посмотрим на то, как далеко мы ушли от случайного прогноза по качеству:

In [53]:
class Random_Model:
    def __init__(self):
        pass

    def fit(self, df_train, df_valid):
        pass

    def predict(self, df_test):
        return np.random.rand(len(df_test))

In [54]:
model = Random_Model()

y_hat_test = model.predict(test)
print(f"Predicted: y_hat_test.shape = {y_hat_test.shape}")

    
# Compute metrics on test
compute_metrics(y_test, y_hat_test, q_test)

Predicted: y_hat_test.shape = (37098,)
metric = NDCG:top=5;type=Exp score = 0.226
metric = NDCG:top=10;type=Exp score = 0.264


Итого, метрики нашей модели на тесте:
```
metric = NDCG:top=5;type=Exp score = 0.461
metric = NDCG:top=10;type=Exp score = 0.494
```
Для случайной модели:
```
NDCG:top=5;type=Exp score = 0.226
NDCG:top=10;type=Exp score = 0.264
```
Видим, что модель получилась сильно лучше случайной.

In [65]:
model.model.save_model("catboost_model.cbm")